In [31]:
# coding=UTF-8 
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
class ResNet(nn.Module):
    def __init__(self, model):
        super(ResNet, self).__init__()
        self.resnet_layer = nn.Sequential(*list(model.children())[:-1])#去掉预训练resnet模型的后1层(fc层)
        self.Linear_layer = nn.Linear(2048, 4)#分类层

    def forward(self, x):
        x = self.resnet_layer(x)
        x = x.view(x.size(0), -1) 
        x = self.Linear_layer(x)
        return x

In [33]:
class FaceAPI(object):
    def __init__(self, model_path):
        resnet = models.resnet50(pretrained=True)
        self.model = ResNet(resnet)#加载一下之前训练好的
        self.model.load_state_dict(torch.load (model_path, map_location='cpu'))
        self.model.eval()#只能预测，不能训练，所以要加这一句evaluate的简写，不然的话，如果里面有dropout,那么预测的时候也会有dropout，我们不希望这样
                        #这是别人定义好的函数，resnet本身就有这函数，调用一下这个，它就知道时候预测了，不是训练了，那么它里面的dropout什么的就不会起作用了，
        self.label_dict = {0: 'left', 1: 'right', 2: 'up', 3: 'straight'}
        
    #定义函数的先后顺序没有要求，因为定义的时候还没有调用
    def predict(self, image):#预测
        image = self._preprocess(image)
        output = self.model(image).argmax(dim =1).numpy()[0]
        return self.label_dict[output], np.transpose(image.numpy()[0],(1,2,0))#第二个返回值是为了确认一下，是否变为灰度图了
    
    def _preprocess(self, image):#处理图片
        image = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)
        image = cv2.resize(image, (224,224))
        image = torch.tensor(np.transpose(image, (2, 0, 1)),dtype = torch.float32).view(1, 3, 224, 224)
        return image

In [34]:
tmp = FaceAPI("D:\\workshop\\test1\\resnet50_face.pt")

In [35]:
cap=cv2.VideoCapture(0)
while True:    
    #从摄像头读取图片    
    sucess,img=cap.read()
    text = tmp.predict(img)[0]
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, text, (200,100), font, 2, (0,255,0), 3) 
    cv2.imshow("img",img)   
    #保持画面的持续。    
    k=cv2.waitKey(1)    
    if k == 27:        
        #通过esc键退出摄像        
        cv2.destroyAllWindows()        
        break    
    elif k==ord("s"):        
        #通过s键保存图片，并退出。        
        cv2.imwrite("image2.jpg",img)
        cv2.destroyAllWindows()
        break

#关闭摄像头
cap.release()
    